In [11]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# unit sphere constraint
u=5
p=floor(Int64,n/u)+1

I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
f=sum(generate_random_poly(I[j]) for j in 1:p)



g=-(x.^2).+(1/u)



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/7)

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)

randx=2*rand(n).-1
randx=randx./sqrt(u)

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make constraints feasible
    end
end

l=length(h)
println("Number of equality constraints: l=",l)
println("====================")



***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=5
Number of equality constraints: l=1


In [12]:
k=2

println("Relaxed order: k=",k)
println("====================")

t=1

println("Sparse order: t=",t)
println("====================")

Relaxed order: k=2
Sparse order: t=1


In [13]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [14]:
using TSSOS


@time begin
    opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS="block",CTP=false);
    #opt,sol,data=cs_tssos_higher!(data,TS="block");
end

------------------------------------------------------
The clique sizes of varibles:
[5]
[1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 91              
  Cones                  : 0               
  Scalar variables       : 22              
  Matrix variables       : 8               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00  

(-0.5331953003358436, nothing, TSSOS.mdata_type(5, 0, 6, [2, 2, 2, 2, 2, 2], SparseMatrixCSC{UInt8,UInt32}[
  [1,  1]  =  0x02
  [1,  2]  =  0x01
  [2,  2]  =  0x01
  [1,  3]  =  0x01
  [3,  3]  =  0x01
  [1,  4]  =  0x01
  [4,  4]  =  0x01
  [1,  5]  =  0x01
  [5,  5]  =  0x01
  [2,  6]  =  0x02
  [2,  7]  =  0x01
  [3,  7]  =  0x01
  [2,  8]  =  0x01
  [4,  8]  =  0x01
  [2,  9]  =  0x01
  [5,  9]  =  0x01
  [3, 10]  =  0x02
  [3, 11]  =  0x01
  [4, 11]  =  0x01
  [3, 12]  =  0x01
  [5, 12]  =  0x01
  [4, 13]  =  0x02
  [4, 14]  =  0x01
  [5, 14]  =  0x01
  [5, 15]  =  0x02, 
  [1, 1]  =  0x02, 
  [2, 1]  =  0x02, 
  [3, 1]  =  0x02, 
  [4, 1]  =  0x02, 
  [5, 1]  =  0x02, 
  [5, 1]  =  0x02], Array{Float64,1}[[0.6626117533412383, 0.6891980419938131, -0.7234038092961699, 0.1292320020044846, 0.9765518237575104, -0.25102281269110494, -0.8285319875811972, -0.6712743356044268, 0.9089764812642476, -0.02446544917440807, -0.49353905135950304, -0.7134180339656364, -0.0027361971815560437, 0.0

In [15]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val1=ctpPOP.POP_mix_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[5]
[1]
------------------------------------------------------
  Number of cliques: p=1
  Largest clique size: u=5


  Largest constant trace: a=3.0
  Number of blocks: omega=12
  Number of equality consraints: zeta=162
  Size of the largest block: s^max=16
Modeling time:
  0.957637 seconds (718.03 k allocations: 36.482 MiB, 7.41% gc time)
iter=1   val=-2.55467284216763   gap=0.0   feas=3.3412683530552747
iter=2   val=-0.8515576140558767   gap=0.5545826935318479   feas=0.6730752570463944
iter=4   val=-0.7420368265191777   gap=0.447184466498697   feas=0.8486433338158623
iter=8   val=-0.6017484633132005   gap=0.04973509403365674   feas=0.45037169048393055
iter=16   val=-0.5413682688098325   gap=0.23337285140490188   feas=0.24772082453685051
iter=32   val=-0.6199109787246758   gap=0.19923853291857402   feas=0.14428843356782983
iter=64   val=-0.5274648062330041   gap=0.05029826670988737   feas=0.06633644438971276
iter=128   val=-0.5216538627623573   gap=0.06605660386267483   feas=0.023432846172524113
iter=256   val=-0.5343145844210737   gap=0.06378578170287139   feas=0.013176694463712788
iter=429   val=-

-0.5294003349478444

In [16]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        1
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0